In [ ]:
import hnswlib
from PIL import Image
from ShufflePatchModel16 import ShufflePatchFeatureExtractor
from VggFeatureExtractor import VggFeatureExtractor
from glob import glob
import random
import numpy as np
import ipyplot
import math
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from itertools import compress
import pickle
import networkx as nx
import cv2

%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
window_size = 32
stride = 24
kp_margin = 16 # keypoint detector has a margin around image where it can not find keypoints
n_clusters = 1000

cnn = VggFeatureExtractor()

def extract_windows(frame, pos, window_size):
    windows = np.empty((len(pos), window_size, window_size, 3), dtype=np.uint8)

    for i in range(len(pos)):
        windows[i] = extract_window(frame, pos[i], window_size)

    return windows


def extract_window(frame, pos, window_size):
    half_w = window_size/2.0

    top_left = [int(round(pos[0]-half_w)), int(round(pos[1]-half_w))]
    bottom_right = [top_left[0]+window_size, top_left[1]+window_size]

    return frame[top_left[0]:bottom_right[0], top_left[1]:bottom_right[1]]

- Extract all patches from dataset and generate features
- Cluster features

In [ ]:
image_files = glob("dataset_100/train/*/*.jpg")

X = []

for idx, image_file in enumerate(image_files):
    print(idx, image_file)
    pil_image = Image.open(image_file).convert('RGB')
    pil_image = pil_image.resize((int(round(pil_image.size[0]/3)), int(round(pil_image.size[1]/3))))
    image = np.array(pil_image)

    grid = (math.floor((image.shape[0] - (window_size - stride)) / stride), math.floor((image.shape[1] - (window_size - stride)) / stride))
    margin = ((image.shape[0] - grid[0] * stride)/2, (image.shape[1] - grid[1] * stride)/2)
    points = [(margin[0]+y*stride+stride/2,margin[1]+x*stride+stride/2) for y in range(grid[0]) for x in range(grid[1])]
    
    print('len(points)', len(points))
    
    patches = extract_windows(image, points, 32)

    windows = patches.astype(np.float64)

    feats = cnn.evalRGB(windows)
    feats = feats.reshape((windows.shape[0], 512))
    X.extend(list(feats))


print("Clustering with KMeans")
clusters = KMeans(n_clusters=n_clusters, verbose=True)
clusters.fit(np.array(X, dtype=np.float32))

cluster_count = len(np.unique(clusters.labels_))
print('cluster_count', cluster_count)

pickle.dump(clusters, open("clusters.pkl", "wb"))
    

In [ ]:
# select_count = 10

# select_ids = random.sample(patch_dict.keys(), select_count)
# #print(select_ids)
# select_feats = index.get_items(select_ids)
# #print(len(select_feats))
# nn_ids, nn_dis = index.knn_query(select_feats, 10)

# for i in range(select_count):
#     patch = patch_dict.get(select_ids[i])
#     nn_patches = [patch_dict.get(q) for q in nn_ids[i]]
#     nn_dis_labels = [q for q in nn_dis[i]]

#     ipyplot.plot_images([patch], ['patch'])
#     ipyplot.plot_images(nn_patches, nn_dis_labels)

- Save Cluster Model
- Perform random walks from patch to  patch
- generate graph of adjacencies and frequencies

## Build adjacency graph with patch walk

In [ ]:
def salient_grid_locations(image, stride, offsets, orb):

    kp = orb.detect(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), None)

    grid = set()

    for k in kp:
        p = (k.pt[1], k.pt[0]) 
        g = (int(math.floor((p[0]-offsets[0])/stride)), int(math.floor((p[1]-offsets[1])/stride)))
        grid.add(g)
    
    return grid


def get_rad_grid(grid_pos, rad, grid_shape):

    top_left = (grid_pos[0]-rad, grid_pos[1]-rad)

    res = []

    for i in range(2*rad+1):
        p = (top_left[0]+i, top_left[1])
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)
 
    for i in range(2*rad+1):
        p = (top_left[0]+i, top_left[1]+(2*rad))
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)

    for i in range(2*rad-1):
        p = (top_left[0], top_left[1]+(i+1))
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)

    for i in range(2*rad-1):
        p = (top_left[0]+(2*rad), top_left[1]+(i+1))
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)

    return res





def add_image_adjacencies(image_file, clusters, feature_extractor, adjacency_graph, orb):
    print("add_image_adjacencies", image_file)

    pil_image = Image.open(image_file).convert('RGB')
    pil_image = pil_image.resize((int(round(pil_image.size[0]/3)), int(round(pil_image.size[1]/3))))
    image = np.array(pil_image)

    margin = max(window_size, kp_margin*2)
    grid_shape = (math.floor((image.shape[0] - margin) / stride), math.floor((image.shape[1] - margin) / stride))
    offsets = (round((image.shape[0] - grid_shape[0] * stride)/2), round((image.shape[1] - grid_shape[1] * stride)/2))

    grid_locations_set = salient_grid_locations(image, stride, offsets, orb)
    grid_locations_list = list(grid_locations_set)
    
    points = [(y*stride + stride/2 + offsets[0], x*stride + stride/2 + offsets[1]) for (y,x) in grid_locations_list]
    
    print('salient grid locations', grid_shape, len(grid_locations_list), 'of', grid_shape[0] * grid_shape[1])
        
    patches = extract_windows(image, points, window_size)
    windows = patches.astype(np.float64)

    feats = cnn.evalRGB(windows)
    feats = feats.reshape((windows.shape[0], 512))

    grid_cluster_ids = clusters.predict(feats)
    
    adjacency_count = 0
    
    for location in grid_locations_list:
        cluster_id = grid_cluster_ids[grid_locations_list.index(location)]
        
        adjacent_locations = get_rad_grid(location, 1, grid_shape)
        
        adjacent_cluster_ids = [grid_cluster_ids[grid_locations_list.index(g)] for g in adjacent_locations if g in grid_locations_set]
        
#         print(location, adjacent_locations, cluster_id, adjacent_cluster_ids)
        
        for adjacent_cluster_id in adjacent_cluster_ids:
            if adjacency_graph.has_edge(cluster_id, adjacent_cluster_id):
                adjacency_graph[cluster_id][adjacent_cluster_id]['weight'] += 1
            else:
                adjacency_graph.add_edge(cluster_id, adjacent_cluster_id, weight=1)
            adjacency_count += 1
        
    print('adjacency_count', adjacency_count)

In [ ]:
image_files = glob("dataset_100/train/*/*.jpg")

clusters = pickle.load(open("clusters.pkl", "rb"))
feature_extractor = VggFeatureExtractor()
orb = cv2.ORB_create(nfeatures=100000, fastThreshold=7)

adjacency_graph = nx.Graph()

for i in range(len(image_files)):
    add_image_adjacencies(image_files[i], clusters, feature_extractor, adjacency_graph, orb)

nx.write_gpickle(adjacency_graph, "adjacency_graph.gpickle")
    
nx.draw(adjacency_graph)
plt.show()

There is a graph containing overlapping communities
Given a sequence of observations, find communities that likely generated sequence